In [9]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [17]:
driver = uc.Chrome()

In [18]:
driver.get("https://www.card-gorilla.com/chart/top100?term=weekly&gad_source=1&gclid=Cj0KCQiAgJa6BhCOARIsAMiL7V9t0g1QuBq8YWrlp5yBmxjqLg7i5L7_FMGu1xazVXfuFFek5yc2o_EaAkK-EALw_wcB")

In [ ]:
from tqdm import tqdm
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


이용조건_하나로 = []
name_1 = []
company_1 = []
benefit_1 = []

for n in tqdm(range(1, len(driver.find_elements(By.CSS_SELECTOR, "p.card_name")))):
    try:
        # 카드 상세 페이지로 이동
        driver.find_elements(By.CSS_SELECTOR, "p.card_name")[n].click()
        time.sleep(1)
        
        # 카드 정보 수집
        name = driver.find_elements(By.CSS_SELECTOR, "strong.card")[0].text  # 카드 이름
        company = driver.find_elements(By.CSS_SELECTOR, "p.brand")[0].text  # 카드사 이름
        benefit = driver.find_elements(By.CSS_SELECTOR, "div.lst.bene_area")[0].text  # 총 혜택
        time.sleep(1)
        
        name_1.append(name)
        company_1.append(company)
        benefit_1.append(benefit)
        
        # 이용 조건 수집
        이용조건_1 = []
        for i in range(len(driver.find_elements(By.CSS_SELECTOR, "p.txt1"))):
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, "p.txt1")  # 요소 새로 찾기
                
                # 스크롤로 요소를 화면 중앙에 위치
                target = elements[i]
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", target)
                
                # 클릭 가능 상태 대기
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "p.txt1")))
                
                # 클릭 실행
                target.click()
                time.sleep(1)
                
                # 이용 조건 가져오기
                이용조건 = driver.find_elements(By.CSS_SELECTOR, "div.in_box")[1].text
                이용조건_1.append(이용조건)
                
                # 새로고침
                driver.refresh()
                time.sleep(1)
            except Exception as e:
                print(f"[이용 조건 수집 오류] index {i}: {e}")
                continue
        
        이용조건_하나로1 = "\n\n".join(item if item else "' '" for item in 이용조건_1)
        이용조건_하나로.append(이용조건_하나로1)
        
        # 이전 페이지로 돌아가기
        driver.execute_script("window.history.go(-1)")
        time.sleep(1)
    
    except Exception as e:
        print(f"[카드 정보 수집 오류] index {n}: {e}")
        driver.execute_script("window.history.go(-1)")  # 오류 시 이전 페이지로 복귀
        time.sleep(1)
        continue

In [ ]:
data = {
    '카드명': name_1,
    '카드회사': company_1,
    '혜택_요약': benefit_1,
    '혜택': 이용조건_하나로
}

# DataFrame 생성
df = pd.DataFrame(data)
df

In [ ]:
df.to_csv('카드혜택.csv', index = False)